In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import pandas as pd

In [ ]:
# Criando Navegador e maximizando a tela
navegador = webdriver.Chrome()
navegador.maximize_window()
navegador.get('https://sistemacbm.com/#/dashboard/')
time.sleep(15)

In [ ]:
#preechendo campo de usuario com cpf
elemento = navegador.find_element(By.XPATH, '//*[@id="novo_login-columns"]/div[2]/div/section/form/div[1]/div/input')
elemento.send_keys('user_name')

In [ ]:
#preechendo campo de senha com password 
elemento = navegador.find_element(By.XPATH, '//*[@id="novo_login-columns"]/div[2]/div/section/form/div[2]/div/input')
elemento.send_keys('password')
time.sleep(5)

In [ ]:
#clicando no botao entrar 
elemento = navegador.find_element(By.XPATH, '//*[@id="novo_login-columns"]/div[2]/div/section/form/div[3]/button')
elemento.click()
time.sleep(20)

In [ ]:
# lendo a base de dados e criando lista vazia 
dados = pd.read_excel("base.xlsx")
colunas = ['Placa','Proprietário', 'CPF/CNPJ Proprietário']
placas = dados[colunas]
lista_planilha = []

In [ ]:
for placa in placas.Placa.head(5):
    # clicando em autorizacao
    elemento = navegador.find_element(By.XPATH, '//*[@id="main"]/div[2]/div/section/div[1]/div[1]/div/article[1]/aside/ul[1]/li[1]/a[1]')
    navegador.execute_script("arguments[0].scrollIntoView(false);", elemento) 
    time.sleep(5)
    elemento.click()
    time.sleep(6)
    
    # clicando em filtrar
    elemento = navegador.find_element(By.TAG_NAME, 'button')
    time.sleep(2)
    elemento.click()
    time.sleep(5)

    # inserindo digitos alfanumericos da placa
    elemento = navegador.find_element(By.XPATH, '//*[@id="main"]/div[2]/div/div/form/div[1]/div[4]/div/div/input')
    time.sleep(5)
    
    # limpar o campo de texto e escrevenver novo dado
    elemento.clear()
    time.sleep(5)
    elemento.send_keys(placa)
    time.sleep(5)

    # clicando no botao de filtrar
    
    elemento = navegador.find_element(By.XPATH, '//*[@id="main"]/div[2]/div/div/form/div[3]/div/button[1]/span[2]')
    elemento.click()
    time.sleep(3)

    # diminuindo a tela e clicando na lupa
    time.sleep(2)
    elemento = navegador.find_element(By.XPATH, '//*[@id="main"]/div[2]/div/div/div/table/tbody/tr/td[12]')
    navegador.execute_script("arguments[0].scrollIntoView(true);", elemento) 
    time.sleep(2)
    elemento.click()

    # extraindo fonte html da page atual com Selenium e BeautifulSoup
    time.sleep(5)
    page_content = navegador.page_source
    soup = BeautifulSoup(page_content, 'html.parser')

    texto_page = soup.get_text()
    lista = texto_page.split()
    indice = lista.index('CPF/CNPJ')
    
    #nome_loja = lista[lista.index('PagadorEditarObservação') + 1 :238]
    nome_loja = navegador.find_elements(By.CLASS_NAME, 'leitura')[11].text
   
    #nome_loja = ' '.join(nome_loja)    
    cpf_cnpj = lista[indice + 3]
    nome = lista[indice + 8] + ' ' + lista[indice + 9] + ' ' + lista[indice + 10] + ' ' + lista[indice + 11] + ' ' + lista[indice + 12] + ' ' + lista[indice + 13]
    nome = nome.split('Cidade')[0]
    data = lista[112]
    
    #voltando ao menu home    
    elemento = navegador.find_element(By.XPATH, '//*[@id="main-sidebar"]/nav/ul[1]/li/a')
    elemento.click()
    time.sleep(3)
    
    print(data, placa, cpf_cnpj, nome, nome_loja)
    lista_planilha.append([data, placa, nome, nome_loja])
    

In [ ]:
# tamanho da lista estraida da automaçao
len(lista_planilha)

In [ ]:
# Criando o DataFrame apos a raspagem de dados 
tabela = pd.DataFrame(lista_planilha, columns=['data', 'placa', 'nome', 'nome_loja'])

In [ ]:
display(tabela.head())

In [ ]:
# Classificando a tabela pela coluna 'Nome' em ordem alfabetica
tabela_sorted = tabela.sort_values(by='data', ignore_index=True)

In [ ]:
# criando uma coluna de categoria de 'NOVOS/USADOS'
tabela_sorted['USADOS/NOVOS'] = tabela_sorted['placa'].apply(lambda x: 'NOVOS' if 'SN' in x else 'USADOS').str.lower()

In [ ]:
display(tabela_sorted.head())

In [ ]:
tabela_sorted.nome_loja.value_counts()

In [ ]:
# Salvando tabela em excel
tabela_sorted.to_excel('output.xlsx', index=False)

In [ ]:
# Finalizando o navegador 
navegador.close()